In [38]:
import nltk

In [39]:
filename = 'data/feedback_cs2012_1.txt'

In [40]:
fp = open(filename, 'r')

In [41]:
all_content = fp.read()

In [42]:
all_content

"Lectures were really good. There were lot of people who came their without any Java knowledge and yet you were very supportive. 5/5 for me.\n\nIt's really good that the lecturer explains most of the concepts using examples. It helps us to understand them better. But if you are sitting at the back, you can't properly see what's on the board because the board is too small and the lecturer writes in small letters due to that. On general lectures are understandable and good.\n\nAll the 7 lectures were clearly explained and had a good content also. The way of described the oop concepts were understandable. practical examples that used to explain concepts and exercises which were done during the lecture time was really helped full. Lecture slides were also good and useful.\n\nI like your teaching style.That is really helps us to understand the basic principles.\n\nThe lectures were well organized and detailed. \n\nGreat. Examples and in class activities (writing programms) were very helpful

In [43]:
text = nltk.word_tokenize(all_content)
pos_tagged_text = nltk.pos_tag(text)

In [44]:
 nltk.help.upenn_tagset('NNS')

NNS: noun, common, plural
    undergraduates scotches bric-a-brac products bodyguards facets coasts
    divestitures storehouses designs clubs fragrances averages
    subjectivists apprehensions muses factory-jobs ...


In [45]:
pos_tagged_text

[('Lectures', 'NNS'),
 ('were', 'VBD'),
 ('really', 'RB'),
 ('good', 'JJ'),
 ('.', '.'),
 ('There', 'EX'),
 ('were', 'VBD'),
 ('lot', 'NN'),
 ('of', 'IN'),
 ('people', 'NNS'),
 ('who', 'WP'),
 ('came', 'VBD'),
 ('their', 'PRP$'),
 ('without', 'IN'),
 ('any', 'DT'),
 ('Java', 'NNP'),
 ('knowledge', 'NN'),
 ('and', 'CC'),
 ('yet', 'RB'),
 ('you', 'PRP'),
 ('were', 'VBD'),
 ('very', 'RB'),
 ('supportive', 'JJ'),
 ('.', '.'),
 ('5/5', 'CD'),
 ('for', 'IN'),
 ('me', 'PRP'),
 ('.', '.'),
 ('It', 'PRP'),
 ("'s", 'VBZ'),
 ('really', 'RB'),
 ('good', 'JJ'),
 ('that', 'IN'),
 ('the', 'DT'),
 ('lecturer', 'NN'),
 ('explains', 'VBZ'),
 ('most', 'JJS'),
 ('of', 'IN'),
 ('the', 'DT'),
 ('concepts', 'NNS'),
 ('using', 'VBG'),
 ('examples', 'NNS'),
 ('.', '.'),
 ('It', 'PRP'),
 ('helps', 'VBZ'),
 ('us', 'PRP'),
 ('to', 'TO'),
 ('understand', 'VB'),
 ('them', 'PRP'),
 ('better', 'JJR'),
 ('.', '.'),
 ('But', 'CC'),
 ('if', 'IN'),
 ('you', 'PRP'),
 ('are', 'VBP'),
 ('sitting', 'VBG'),
 ('at', 'IN'),
 ('

In [47]:
nltk.tag.tuple2str(('any', 'DT'))

'any/DT'

In [48]:
string_pos_text = [ nltk.tag.tuple2str(tup) for tup in pos_tagged_text]

In [49]:
one_line = ' '.join(string_pos_text)

In [50]:
out_filename = 'data/tagged_corpus.txt'
wfp = open(out_filename, 'w')
wfp.write(one_line)
wfp.close()